In [1]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')  # on définit les paramètres de SimpleImputer
# on entraine ce modèle sur un tableau (3,3) de meme dim en colonne que celui qui nous importe ensuite (PAS SUR )
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])  
SimpleImputer()


SimpleImputer()

In [2]:
X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9], [3, 2, np.nan]] # voici le talbeau à traiter au final (4, 3)
print(imp_mean.transform(X)) # on le transforme par l'entrainement fait sur SimpleImpute 

[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]
 [ 3.   2.   6. ]]


In [3]:
# MAIS PB SI DATASET AVEC DES NOMS DE COLONNES, CES NOMS DISPARAISSENT CAR TRAITEMENT D UN TABLEAU NDARRAY

## SimpleImputer par entrainement sur un tableau (n, p ) - video 22-2

In [4]:
import numpy as np
from sklearn.impute import SimpleImputer

In [5]:
X = np.array([[10, 3], [0,4], [5,3], [np.nan, 3]])   # tableau (4, 2)

In [6]:
imputer = SimpleImputer(missing_values = np.nan,     # missing_values = VALEUR A NETTOYER = np.nan ou -99, -1,  -99999, inf etc ce qu'on veut !
              strategy = 'mean', copy=False)            # strategy = 'mean' , 'median' , 'most_frequent' ou 'constante' avec , fill_value='xx' pour constante  
              

In [7]:
imputer.fit_transform(X)  # puis entrainer notre imputer sur X
# ici remplace le np.nan par la moyenne de sa colonne ( 10, 0, 5) soit nan devient 5

array([[10.,  3.],
       [ 0.,  4.],
       [ 5.,  3.],
       [ 5.,  3.]])

In [8]:
X

array([[10.,  3.],
       [ 0.,  4.],
       [ 5.,  3.],
       [ 5.,  3.]])

### ok ensuite pour traiter un autre tableau (x lignes, meme nbre de colonnes ) = (m, p) que le fit_transforme !

In [9]:
# ensuite on peut utiliser notre imputer sur un autre X_test par exemple
X_test = np.array([[1, 2], [3, 4], [5, 6], [np.nan, np.nan], [1, 2]])   # tableau (5, 2)
imputer.transform(X_test)   # pas avec fit mais seulement transform() cette fois-ci car fit déjà fait
# ATTENTION
# [np.nan, np.nan] de vient ici [ 5, 3.25 ] qui sont les moyennes calculées sur le fit_transform !! et pas sur X_test
# car on ne peut utiliser des données des calculs nouvelles pour ne pas polluer notre train-test ou train_val ensuite

array([[1.  , 2.  ],
       [3.  , 4.  ],
       [5.  , 6.  ],
       [5.  , 3.25],
       [1.  , 2.  ]])

### B - KNNImputer K-nearest-Neighbors Imputer = remplace par les + proches voisins

In [10]:
from sklearn.impute import KNNImputer

In [11]:
X = np.array([[1, 100], [2, 30], [3, 15], [np.nan, 20]])   # tableau (4, 2) 1,2,3=classe et 100, 30, 15 = prix titanic

In [12]:
imputer = KNNImputer(n_neighbors = 1)
imputer.fit_transform(X)  # remplace nan par 3 car classe 3 à 15€ le + proche

array([[  1., 100.],
       [  2.,  30.],
       [  3.,  15.],
       [  3.,  20.]])

### très utile à utiliser avec un GridsearchCV sur vrai dataset Titanic

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV , train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.impute import KNNImputer
import seaborn as sns

In [14]:
titanic = sns.load_dataset('titanic')
X = titanic[['pclass', 'age']]
y = titanic['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [15]:
model = make_pipeline(KNNImputer(), SGDClassifier())  # SGDClassifier pour faire de laclassification au lieu de fonct° Pandas
# car impossible un classifier de Pandas dans 1 pipeline
model

Pipeline(steps=[('knnimputer', KNNImputer()),
                ('sgdclassifier', SGDClassifier())])

In [16]:
params = {
    'knnimputer__n_neighbors' : [1, 2, 3, 4]    # pas de params sur SGDClassifier pour faire simple ici
}

In [17]:
grid = GridSearchCV(model, param_grid=params, cv=5) # on créé notre grille de recherche de best_params

In [18]:
grid.fit(X_train, y_train)  # on entraine notre grille

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('knnimputer', KNNImputer()),
                                       ('sgdclassifier', SGDClassifier())]),
             param_grid={'knnimputer__n_neighbors': [1, 2, 3, 4]})

In [19]:
grid.best_params_

{'knnimputer__n_neighbors': 2}

### C - MissingIndicator = indique la présence ou non de valeur manquantes dans une variable

In [20]:
from sklearn.impute import MissingIndicator

In [21]:
Z = np.array([[1, 100], [2, 30], [3, 15], [np.nan, np.nan]])   # tableau (4, 2) 1,2,3=classe et 100, 30, 15 = prix titanic
# et (nan, nan) double inconnu = membre d'équipage probablement sans classe ni tarif

In [22]:
imputer = MissingIndicator()
imputer.fit_transform(Z)  

array([[False, False],
       [False, False],
       [False, False],
       [ True,  True]])

In [23]:
from sklearn.pipeline import make_union
# pour traiter en // SimpleImputer et         ET concaténer les résultats en 1 seul tableau
pipeline = make_union(SimpleImputer(strategy = 'constant', fill_value = -99),   # -99 = code classe new passager = équipage
                      MissingIndicator())   # création tableau (4, 2 )x2 -> 1 pour le SimpleI et 1 pour le MissingI
pipeline.fit_transform(Z)
                      

array([[  1., 100.,   0.,   0.],
       [  2.,  30.,   0.,   0.],
       [  3.,  15.,   0.,   0.],
       [-99., -99.,   1.,   1.]])